# 🧼 Limpieza de Datos - Calidad del Agua del Río Cauca

Este notebook documenta el proceso de limpieza del dataset descargado desde [datos.gov.co](https://www.datos.gov.co/), específicamente sobre la calidad del agua del Río Cauca.

## Objetivos:
- Estandarizar columnas
- Eliminar columnas completamente vacías
- Convertir fechas
- Limpiar y convertir columnas numéricas con símbolos, notación científica o errores


In [ ]:
import pandas as pd
import numpy as np
import re

# Cargar el dataset desde carpeta raw
df = pd.read_csv("../data/raw/Calidad_del_agua_del_Rio_Cauca_20250605.csv", encoding="utf-8")
df.head()


In [ ]:
# Convertir columna de fechas
df['FECHA DE MUESTREO'] = pd.to_datetime(df['FECHA DE MUESTREO'], errors='coerce')


In [ ]:
# Eliminar columnas sin datos
df = df.dropna(axis=1, how='all')
print(f"Número de columnas restantes: {df.shape[1]}")


In [ ]:
# Detectar columnas tipo object que podrían ser numéricas
possible_numeric_cols = [col for col in df.select_dtypes(include='object').columns if col not in ['ESTACIONES']]

# Función para limpiar valores
def clean_numeric(value):
    if pd.isnull(value):
        return np.nan
    if isinstance(value, str):
        value = value.replace(',', '.')
        value = re.sub(r'[<>*]', '', value)
        value = re.sub(r'(\d+(\.\d+)?)[*]10E(\d+)', r'\1E\3', value)
        try:
            return float(value)
        except ValueError:
            return np.nan
    return value

# Aplicar limpieza a todas las columnas identificadas
for col in possible_numeric_cols:
    df[col] = df[col].apply(clean_numeric)


In [ ]:
# Guardar en carpeta cleaned
import os
os.makedirs("../data/cleaned", exist_ok=True)
df.to_csv("../data/cleaned/agua_rio_cauca_limpio.csv", index=False)
print("✅ Archivo limpio guardado correctamente.")
